In [1]:
import pyspark.sql.functions as F
from pyspark.sql.types import *

In [45]:
folder_location = "abfss://raw@<<storage_account_name>>.dfs.core.windows.net/missouri/agent_interaction_type_query/*/*/*/"
silver_folder = "abfss://silver@<<storage_account_name>>.dfs.core.windows.net/mo/delta/genesys/agent_interaction_type_query_sparkdemo"

In [12]:
input_schema = spark.read.format("json").load(folder_location).schema

In [39]:
df = spark.readStream.format("json").schema(input_schema).load(folder_location)
df.createOrReplaceTempView("df")

In [40]:
%%sql

create or replace TEMPORARY view df1 as 
select exploded.* from 
(select explode(results) as exploded from df) as x

In [41]:
%%sql

create or replace TEMPORARY view df2 as 
select dataexploded.interval as data_interval,
    explode(dataexploded.metrics) as metricsexploded, 
    group1.*
from 
(select explode(data) as dataexploded, group as group1 from df1) as x

In [42]:
%%sql

create or replace TEMPORARY view df3 as 
select data_interval, metric, stats.*, direction, interactionType, mediaType, userId
from
(select data_interval, metricsexploded.*, direction, interactionType, mediaType, userId from df2)

In [43]:
output_df = spark.sql("select * from df3;")

In [47]:
streamingQuery = (output_df
    .writeStream
    .format("delta")
    .option("checkpointLocation", silver_folder + "/checkpoint")
    .option("mergeSchema", True)
    .outputMode("append")
    .queryName("agent_interaction_type_query_stream")
    .trigger(once=True)
    .start(silver_folder)
)

In [48]:
for s in spark.streams.active:
    print(s.name)
    print(s.status)
    print(s.lastProgress)
    s.awaitTermination()

In [49]:
%%sql

create database if not exists Missouri_ADLS;

In [50]:
%%sql

use database Missouri_ADLS

In [51]:
%%sql

create table if not exists Missouri_ADLS.agent_interaction_type_query
using delta
location 'abfss://silver@<<storage_account_name>>.dfs.core.windows.net/mo/delta/genesys/agent_interaction_type_query_sparkdemo'